In [104]:
import pandas as pd
import numpy as np
import os

import re
import emoji
from konlpy.tag import Okt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader -> datasets 라이브러리랑 충돌

In [ ]:
from dotenv import load_dotenv
import boto3
from io import BytesIO

load_dotenv()
aws_access_key = os.getenv("aws_accessKey")
aws_secret_key = os.getenv("aws_secretKey")
bucket_name = 'flexmatch-data'
region_name = 'ap-northeast-2'

s3 = boto3.client('s3',
                aws_access_key_id=aws_access_key,
                aws_secret_access_key=aws_secret_key,
                region_name=region_name)

profile_data_prefix = f'instagram-data/tables/EXTERNAL_2_RECENT_USER_INFO_MTR/year=2025/week=30/'
media_data_prefix = f'instagram-data/tables/EXTERNAL_2_BY_USER_ID_MEDIA_DTL_INFO/year=2025/week=30/'

# profile 파일 목록
profile_response = s3.list_objects_v2(Bucket=bucket_name, Prefix=profile_data_prefix)
profile_keys = [obj['Key'] for obj in profile_response.get('Contents', []) if obj['Key'].endswith('.parquet')]

# media 파일 목록
media_response = s3.list_objects_v2(Bucket=bucket_name, Prefix=media_data_prefix)
media_keys = [obj['Key'] for obj in media_response.get('Contents', []) if obj['Key'].endswith('.parquet')]

# 읽기
profile_dfs = []

for key in profile_keys:
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    body = obj['Body'].read()
    df = pd.read_parquet(BytesIO(body))
    profile_dfs.append(df)

new_profile_data = pd.concat(profile_dfs, ignore_index=True)

media_dfs = []

for key in media_keys:
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    body = obj['Body'].read()
    df = pd.read_parquet(BytesIO(body))
    media_dfs.append(df)

new_media_data = pd.concat(media_dfs, ignore_index=True)


In [ ]:
# os.chdir("C:/Users/ehddl/Desktop/업무/code/sns-categorizer/")

# new_data = pd.read_parquet("media_data_sample.parquet")
# new_data_profile = pd.read_parquet("C:/Users/ehddl/Downloads/merged_data.parquet")

# new_data_profile = new_data_profile[['acnt_id', 'acnt_nm', 'acnt_sub_nm', 'intro_txt']]
# new_data = new_data[['acnt_id', 'media_cn']]

data preprocessing

In [174]:
new = pd.merge(new_profile_data, new_media_data, on='acnt_id')
new

,acnt_id,acnt_nm,web_addr,acnt_sub_nm,intro_txt,profile_photo_url_addr,acnt_conn_yn_x,category_nm,follower_cnt,follow_cnt,...,media_type_nm,reels_feed_type_nm,media_url_addr,media_unq_url_addr,tmnl_url_addr,reg_dt,media_cn,acnt_conn_yn_y,feed_share_yn,cmnt_actvtn_yn
0,17841400000114584,imyeonduu,https://linkon.id/imyeonduu,연뚜 / 파티쉬,None,https://scontent-nrt1-1.xx.fbcdn.net/v/t51.288...,N,일상,257181,376,...,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/DMeo5aWRYla/,None,2025-07-24T05:26:00+0000,"#초대 여름의 향이 가득한 “오늘밤, 세계에서 이 사랑이 사라진다해도” 관람하고왔어...",N,None,None
1,17841400000114584,imyeonduu,https://linkon.id/imyeonduu,연뚜 / 파티쉬,None,https://scontent-nrt1-1.xx.fbcdn.net/v/t51.288...,N,일상,257181,376,...,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/DI-uJLzJHyG/,None,2025-04-28T06:21:58+0000,"#협찬 저 진짜 야식도 좋아하구 반주도 좋아하는 아이라 수분뺏길일, 부을일이 많단 ...",N,None,None
2,17841400000114584,imyeonduu,https://linkon.id/imyeonduu,연뚜 / 파티쉬,None,https://scontent-nrt1-1.xx.fbcdn.net/v/t51.288...,N,일상,257181,376,...,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/DLmLn8HRG5S/,None,2025-07-02T07:12:45+0000,#협찬\n요즘 타지 다녀올 일이 종종 있는데요 옷에 진심인 저는 구겨지는걸 극도로 ...,N,None,None
3,17841400000114584,imyeonduu,https://linkon.id/imyeonduu,연뚜 / 파티쉬,None,https://scontent-nrt1-1.xx.fbcdn.net/v/t51.288...,N,일상,257181,376,...,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/DMXpmZLJo9O/,None,2025-07-21T12:17:28+0000,#협찬 초경량 포켓사이즈 샤워필터🧖🏻‍♀️\n녹물과 잔류염소를 제거해주는 여행용 샤...,N,None,None
4,17841400000114584,imyeonduu,https://linkon.id/imyeonduu,연뚜 / 파티쉬,None,https://scontent-nrt1-1.xx.fbcdn.net/v/t51.288...,N,일상,257181,376,...,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/DL2NhJapAxz/,None,2025-07-08T12:37:08+0000,#협찬 \n요즘 지인짜 덥죠!\n밤이면 하루종일 시달린 피부 열감이 심한데 그대로 ...,N,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630192,17841474528456437,jun_woo1999,None,황준우,🧸,https://scontent-nrt1-2.xx.fbcdn.net/v/t51.757...,N,일상,1189,113,...,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/DKGhJTfBPmS/,None,2025-05-26T03:33:42+0000,.\n.\n.\n.\n.\n#일상 #ootd #남자모델 #패션모델 #일상스타그램 #...,N,None,None
1630193,17841474528456437,jun_woo1999,None,황준우,🧸,https://scontent-nrt1-2.xx.fbcdn.net/v/t51.757...,N,일상,1189,113,...,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/DMSgmdNB-_G/,None,2025-07-19T12:22:37+0000,Banyan Teer_ Pool Party✨ \nVip #초대 \n.\n.\n.\n...,N,None,None
1630194,17841474528456437,jun_woo1999,None,황준우,🧸,https://scontent-nrt1-2.xx.fbcdn.net/v/t51.757...,N,일상,1189,113,...,IMAGE,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/DKGgcLXBkqI/,None,2025-05-26T03:27:33+0000,None,N,None,None
1630195,17841474528456437,jun_woo1999,None,황준우,🧸,https://scontent-nrt1-2.xx.fbcdn.net/v/t51.757...,N,일상,1189,113,...,IMAGE,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/DKmKeqqh17l/,None,2025-06-07T10:31:21+0000,쁘이~✌️.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\...,N,None,None


In [175]:
def clean_text(text):
    if not isinstance(text, str):
        return ''
    
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [176]:
new['acnt_sub_nm_cleaned'] = new['acnt_sub_nm'].apply(clean_text)
new['intro_txt_cleaned'] = new['intro_txt'].apply(clean_text)
new['media_cn_cleaned'] = new['media_cn'].apply(clean_text)

In [178]:
# new = new[~new.apply(lambda row: row.astype(str).str.strip().eq('').any(), axis=1)]
new = new[new['acnt_sub_nm_cleaned'].ne('') & new['intro_txt_cleaned'].ne('') & new['media_cn_cleaned'].ne('')]

In [ ]:
new_copy = new.copy()
new_copy = new_copy[['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned']].head(300)

In [181]:
num_labels = 26
predict_dataset = Dataset.from_pandas(new_copy)

In [182]:
predict_dataset

Dataset({
    features: ['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned', '__index_level_0__'],
    num_rows: 300
})

single model prediction

In [12]:
# model_name_1 = "kykim/bert-kor-base"
# model_path_1 = "finetune-bert-kor"

# model1 = AutoModelForSequenceClassification.from_pretrained(model_path_1, num_labels=num_labels)
# model1_tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
# model1.eval()

In [13]:
# model_name_2 = "BM-K/KoSimCSE-roberta"
# model_path_2 = "finetune-BM-K"

# model2 = AutoModelForSequenceClassification.from_pretrained(model_path_2, num_labels=num_labels)
# model2_tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta")
# model2.eval()

In [183]:
model_name = "kykim/bert-kor-base"
model_path = "muli-columns-kykim-bert-kor"

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=num_labels)
model_tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# def tokenize_fn_for_prediction(ex):
#     # label 칼럼이 없으므로 text 컬럼만 토큰화 진행
#     return model_tokenizer(ex["media_cn_cleaned"], padding="max_length", truncation=True, max_length=512) # max_length 조정 (128->512)

# predict_dataset = predict_dataset.map(tokenize_fn_for_prediction, batched=True)
# predict_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"]) # 라벨은 없으므로 제외

Map: 100%|██████████| 18282/18282 [00:01<00:00, 12033.08 examples/s]


In [184]:
def tokenize_three_columns(examples):
    combined_texts = [
        f"{acnt} {model_tokenizer.sep_token} {intro} {model_tokenizer.sep_token} {txt}"
        for acnt, intro, txt in zip(
            examples["acnt_sub_nm_cleaned"],
            examples["intro_txt_cleaned"],
            examples["media_cn_cleaned"]
        )
    ]
    
    return model_tokenizer(
        combined_texts,
        padding="max_length",
        truncation=True,
        max_length=512 
    )

In [185]:
predict_dataset = predict_dataset.map(tokenize_three_columns, batched=True)
columns_to_remove = ['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned']
predict_dataset = predict_dataset.remove_columns(columns_to_remove)
predict_dataset.set_format(type="torch", columns=['input_ids', 'attention_mask'])


Map: 100%|██████████| 300/300 [00:00<00:00, 3770.57 examples/s]


In [ ]:
# def tokenize_fn_for_prediction(ex):
#     # label 칼럼이 없으므로 text 컬럼만 토큰화 진행
#     return model1_tokenizer(ex["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128) # max_length 조정 (128->512)

# predict_dataset = predict_dataset.map(tokenize_fn_for_prediction, batched=True)
# predict_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"]) # 라벨은 없으므로 제외

In [ ]:
### roberta 사용 시 token_type_ids가 추가로 있어야 함

# def tokenize_fn_for_prediction(ex):
#     tokenized_output = model2_tokenizer(ex["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128)
#     if "token_type_ids" not in tokenized_output:
#         tokenized_output["token_type_ids"] = [[0] * len(ids) for ids in tokenized_output["input_ids"]]
#     return tokenized_output

# predict_dataset = predict_dataset.map(tokenize_fn_for_prediction, batched=True)
# predict_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", token_type_ids""]) # 라벨은 없으므로 제외

Map: 100%|██████████| 18981/18981 [00:01<00:00, 12261.69 examples/s]


In [186]:
prediction_args = TrainingArguments(
    # output_dir="./prediction_output", # 임시 출력 디렉토리
    per_device_eval_batch_size=16, # 평가 배치 크기
    do_train=False, # 학습은 안 함
    do_predict=True, # 예측만 수행
    report_to="none", # W&B 등 로깅 비활성화
    disable_tqdm=False, # 진행률 바 표시 (기본값 True)
)

# Trainer 인스턴스 생성
trainer = Trainer(model=model, args=prediction_args) # 모델 변경

# 예측 수행
predictions_output = trainer.predict(predict_dataset)

# predictions_output 객체에서 로짓 추출
logits = predictions_output.predictions
print(f"\n예측된 로짓 형태: {logits.shape}")

# 로짓을 확률로 변환 (필요한 경우)
probabilities = torch.softmax(torch.tensor(logits), dim=-1).numpy()

# 가장 높은 확률을 가진 라벨의 인덱스 추출
predicted_class_indices = np.argmax(logits, axis=-1)

# 라벨 인덱스를 실제 라벨 이름으로 매핑
category_labels = [f'{i}' for i in range(num_labels)]
predicted_label_names = [category_labels[idx] for idx in predicted_class_indices]


예측된 로짓 형태: (300, 26)


In [187]:
new_copy['predicted_label'] = predicted_label_names
new_copy['predicted_label'] = new_copy['predicted_label'].astype(int)

In [188]:
label_map = {
 'IT': 0,
 '게임': 1,
 '결혼/연애': 2,
 '교육': 3,
 '다이어트/건강보조식품': 4,
 '만화/애니/툰': 5,
 '문구/완구': 6,
 '미술/디자인': 7,
 '반려동물': 8,
 '베이비/키즈': 9,
 '뷰티': 10,
 '브랜드공식계정': 11,
 '사진/영상': 12,
 '셀럽': 13,
 '스포츠': 14,
 '시사': 15,
 '엔터테인먼트': 16,
 '여행/관광': 17,
 '유명장소/핫플': 18,
 '일상': 19,
 '자동차/모빌리티': 20,
 '짤/밈': 21,
 '취미': 22,
 '패션': 23,
 '푸드': 24,
 '홈/리빙': 25
}

inv_label_map = {v: k for k, v in label_map.items()}
new_copy['single_label'] = new_copy['predicted_label'].map(inv_label_map)

In [189]:
new_copy

,acnt_sub_nm_cleaned,intro_txt_cleaned,media_cn_cleaned,predicted_label,single_label
127,명신91,이명신사임당 수산물스토어 soombisusan,공구를 하다보면 해마다 들어오는 제안들이 비슷비슷한 제품군이에요 여러분들도 공구하는...,4,다이어트/건강보조식품
128,명신91,이명신사임당 수산물스토어 soombisusan,비록 셋 다 육아하느라 여행을 못가고 있지만 우리 마흔살되면 홍콩가자 나이먹고 애낳...,9,베이비/키즈
129,명신91,이명신사임당 수산물스토어 soombisusan,제주도 가기 전 가려던 카페에 스냅 촬영을 한다길래 급하게 예약한 곳 원래 제주 스...,17,여행/관광
130,명신91,이명신사임당 수산물스토어 soombisusan,오늘 미리한 시율이 생파 유치원친구들이랑 꼭 하고싶다해서 초대장 만들어 보냈는데 많...,9,베이비/키즈
131,명신91,이명신사임당 수산물스토어 soombisusan,급 찍게 된 스냅이라 옷도 제대로 못 맞춰 입었지만 그래도 나름 패턴은 맞춤 맘에든...,2,결혼/연애
...,...,...,...,...,...
423,이 상효,주 비전컴퍼니 대표이사 직원이 행복하게 일하고 고객의 미래에 건강을 더하다 비전휘트...,누구보다 일에 쳐 있는 우리 비전인들 또래 친구들보다 동기 지인들보다 하루가 부족해...,19,일상
424,이 상효,주 비전컴퍼니 대표이사 직원이 행복하게 일하고 고객의 미래에 건강을 더하다 비전휘트...,직원이 행복하게 일하고 고객의 미래에 건강을 더하다 5년 전 창업 당시 대부분 비슷...,19,일상
425,이 상효,주 비전컴퍼니 대표이사 직원이 행복하게 일하고 고객의 미래에 건강을 더하다 비전휘트...,차근차근 천천히 하나씩 급하지않게 우리만의 색과 우리들의 이야기로 단단하게 항상 행...,19,일상
426,이 상효,주 비전컴퍼니 대표이사 직원이 행복하게 일하고 고객의 미래에 건강을 더하다 비전휘트...,아빠와아들 붕어빵 말레이시아유학 조호바루 malaysia johorbahru kua...,19,일상


In [152]:
# probs_df = pd.DataFrame(probabilities, columns=[f'prob_{label}' for label in category_labels])
# new = pd.concat([new.reset_index(drop=True), probs_df.reset_index(drop=True)], axis=1)
# new

In [198]:
new_copy_2 = new_copy[['single_label']].reset_index(drop=True)
new_1 = new[['acnt_id', 'acnt_nm', 'acnt_conn_yn_x']].reset_index(drop=True).head(300)

final = pd.concat([new_1, new_copy_2], axis=1)
final

,acnt_id,acnt_nm,acnt_conn_yn_x,single_label
0,17841400012275744,i__luv__yul,N,다이어트/건강보조식품
1,17841400012275744,i__luv__yul,N,베이비/키즈
2,17841400012275744,i__luv__yul,N,여행/관광
3,17841400012275744,i__luv__yul,N,베이비/키즈
4,17841400012275744,i__luv__yul,N,결혼/연애
...,...,...,...,...
295,17841400033674289,hyo___o,N,일상
296,17841400033674289,hyo___o,N,일상
297,17841400033674289,hyo___o,N,일상
298,17841400033674289,hyo___o,N,일상


In [203]:
# final.groupby(['acnt_id', 'acnt_nm'])['single_label'].unique().to_frame()

In [240]:
final.groupby(['acnt_id', 'acnt_nm', 'acnt_conn_yn_x'])['single_label'].value_counts().groupby(level=[0,1]).head(3).reset_index(name='count')

,acnt_id,acnt_nm,acnt_conn_yn_x,single_label,count
0,17841400012275744,i__luv__yul,N,여행/관광,16
1,17841400012275744,i__luv__yul,N,베이비/키즈,15
2,17841400012275744,i__luv__yul,N,일상,4
3,17841400014193921,isapereiranunes,N,엔터테인먼트,16
4,17841400014193921,isapereiranunes,N,교육,8
5,17841400014193921,isapereiranunes,N,여행/관광,8
6,17841400019157809,hipzzi_s2_,N,엔터테인먼트,11
7,17841400019157809,hipzzi_s2_,N,스포츠,8
8,17841400019157809,hipzzi_s2_,N,유명장소/핫플,6
9,17841400019372395,hygge_life_,N,여행/관광,34


In [233]:
top_3_labels = final.groupby(['acnt_id', 'acnt_nm', 'acnt_conn_yn_x'])['single_label'].value_counts().groupby(level=[0,1]).head(3).reset_index(name='count')
top_3_labels_joined = (
    top_3_labels.groupby(['acnt_id', 'acnt_nm', 'acnt_conn_yn_x'])['single_label'].apply(lambda x: '@'.join(x)).reset_index(name='top_3_category')
)

In [234]:
top_3_labels_joined

,acnt_id,acnt_nm,acnt_conn_yn_x,top_3_category
0,17841400012275744,i__luv__yul,N,여행/관광@베이비/키즈@일상
1,17841400014193921,isapereiranunes,N,엔터테인먼트@교육@여행/관광
2,17841400019157809,hipzzi_s2_,N,엔터테인먼트@스포츠@유명장소/핫플
3,17841400019372395,hygge_life_,N,여행/관광@유명장소/핫플
4,17841400021173010,hong.sujin.7,N,셀럽@뷰티@엔터테인먼트
5,17841400032433984,hyejomi,N,다이어트/건강보조식품@일상@스포츠
6,17841400033674289,hyo___o,N,일상@다이어트/건강보조식품@스포츠


In [235]:
main_category = final.groupby(['acnt_id', 'acnt_nm', 'acnt_conn_yn_x'])['single_label'].agg(lambda x: x.value_counts().idxmax()).to_frame().reset_index().rename(columns={'single_label' : 'main_category'})
main_category

,acnt_id,acnt_nm,acnt_conn_yn_x,main_category
0,17841400012275744,i__luv__yul,N,여행/관광
1,17841400014193921,isapereiranunes,N,엔터테인먼트
2,17841400019157809,hipzzi_s2_,N,엔터테인먼트
3,17841400019372395,hygge_life_,N,여행/관광
4,17841400021173010,hong.sujin.7,N,셀럽
5,17841400032433984,hyejomi,N,다이어트/건강보조식품
6,17841400033674289,hyo___o,N,일상


In [ ]:
top_3_labels_joined = top_3_labels_joined.drop(columns=['acnt_id', 'acnt_nm', 'acnt_conn_yn_x'])
final_df = pd.concat([main_category, top_3_labels_joined], axis=1)
final_df

,acnt_id,acnt_nm,acnt_conn_yn_x,main_category,top_3_category
0,17841400012275744,i__luv__yul,N,여행/관광,여행/관광@베이비/키즈@일상
1,17841400014193921,isapereiranunes,N,엔터테인먼트,엔터테인먼트@교육@여행/관광
2,17841400019157809,hipzzi_s2_,N,엔터테인먼트,엔터테인먼트@스포츠@유명장소/핫플
3,17841400019372395,hygge_life_,N,여행/관광,여행/관광@유명장소/핫플
4,17841400021173010,hong.sujin.7,N,셀럽,셀럽@뷰티@엔터테인먼트
5,17841400032433984,hyejomi,N,다이어트/건강보조식품,다이어트/건강보조식품@일상@스포츠
6,17841400033674289,hyo___o,N,일상,일상@다이어트/건강보조식품@스포츠


##### ensemble predict

In [ ]:
# --- 0.4. 각 모델에 맞는 토크나이저 로드 및 텍스트 토큰화 ---

# kykim 모델용 토크나이저 및 데이터셋
model_name_1 = "kykim/bert-kor-base"
model_path_1 = "finetune-bert-kor"
model1 = AutoModelForSequenceClassification.from_pretrained(model_path_1, num_labels=num_labels)
model1.eval()

tokenizer_kykim = AutoTokenizer.from_pretrained(model_name_1)
def tokenize_fn_kykim_for_predict(examples):
    return tokenizer_kykim(examples["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128)

predict_dataset_kykim_tokenized = predict_dataset.map(tokenize_fn_kykim_for_predict, batched=True)
predict_dataset_kykim_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])

# KoSimCSE 모델용 토크나이저 및 데이터셋
model_name_2 = "BM-K/KoSimCSE-roberta"
model_path_2 = "finetune-BM-K"

model2 = AutoModelForSequenceClassification.from_pretrained(model_path_2, num_labels=num_labels)
model2.eval()

tokenizer_kosimcse = AutoTokenizer.from_pretrained(model_name_2)
def tokenize_fn_kosimcse_for_predict(examples):
    tokenized_output = tokenizer_kosimcse(examples["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128)
    if "token_type_ids" not in tokenized_output:
        tokenized_output["token_type_ids"] = [[0] * len(ids) for ids in tokenized_output["input_ids"]]
    return tokenized_output

predict_dataset_kosimcse_tokenized = predict_dataset.map(tokenize_fn_kosimcse_for_predict, batched=True)
predict_dataset_kosimcse_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids"])

# --- 2. Trainer를 사용하여 각 모델의 예측 로짓(Logits) 얻기 ---
# 더미 TrainingArguments (예측용이므로 대부분의 인자는 중요하지 않음)
prediction_args = TrainingArguments(
    output_dir="./voting_prediction_output", # 임시 출력 디렉토리
    per_device_eval_batch_size=16, # 평가 배치 크기
    do_train=False,
    do_predict=True,
    report_to="none", # W&B 등 로깅 비활성화
    remove_unused_columns=False
)

# Model 1 Prediction
trainer1 = Trainer(model=model1, args=prediction_args)
predictions1 = trainer1.predict(predict_dataset_kykim_tokenized)
logits1 = predictions1.predictions # numpy array 형태의 로짓

# Model 2 Prediction
trainer2 = Trainer(model=model2, args=prediction_args)
predictions2 = trainer2.predict(predict_dataset_kosimcse_tokenized)
logits2 = predictions2.predictions


Map: 100%|██████████| 18282/18282 [00:01<00:00, 12330.75 examples/s]


In [ ]:
# --- 3. 로짓(Logits) 또는 확률(Probabilities) 평균내기 ---
# 소프트 보팅은 로짓 또는 softmax 확률을 평균낼 수 있음 (softmax는 비선형 변환이므로 평균 내기 전에 로짓 상태에서 평균 내는 것이 좋음)
# 일반적으로 로짓을 평균내는 것이 더 안정적

# 로짓 평균
ensemble_logits = (logits1 + logits2) / 2 
print(f"\n예측된 로짓 형태: {ensemble_logits.shape}")

# 로짓을 확률로 변환 (필요한 경우)
probabilities = torch.softmax(torch.tensor(ensemble_logits), dim=-1).numpy()

# 평균 로짓에서 최종 예측 클래스 선택
ensemble_predictions = np.argmax(ensemble_logits, axis=-1)

# 라벨 인덱스를 실제 라벨 이름으로 매핑
category_labels = [f'{i}' for i in range(num_labels)]
predicted_label_names = [category_labels[idx] for idx in ensemble_predictions]


예측된 로짓 형태: (18282, 26)


In [19]:
new['predicted_label'] = predicted_label_names
new['predicted_label'] = new['predicted_label'].astype(int)

In [26]:
label_map = {
 'IT': 0,
 '게임': 1,
 '결혼/연애': 2,
 '교육': 3,
 '다이어트/건강보조식품': 4,
 '만화/애니/툰': 5,
 '문구/완구': 6,
 '미술/디자인': 7,
 '반려동물': 8,
 '베이비/키즈': 9,
 '뷰티': 10,
 '브랜드공식계정': 11,
 '사진/영상': 12,
 '셀럽': 13,
 '스포츠': 14,
 '시사': 15,
 '엔터테인먼트': 16,
 '여행/관광': 17,
 '유명장소/핫플': 18,
 '일상': 19,
 '자동차/모빌리티': 20,
 '짤/밈': 21,
 '취미': 22,
 '패션': 23,
 '푸드': 24,
 '홈/리빙': 25
}

inv_label_map = {v: k for k, v in label_map.items()}
new['single_label'] = new['predicted_label'].map(inv_label_map)

In [ ]:
# new.to_excel("C:/Users/ehddl/Desktop/업무/code/sns-categorizer/new_multi-columns.xlsx")